In [ ]:
#https://stackabuse.com/self-organizing-maps-theory-and-implementation-in-python-with-numpy/
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#To understand how the learning rate changes with the decay rate, let's plot the learning rate against various epochs when the initial learning rate is set to one:
epochs = np.arange(0, 50)
lr_decay = [0.001, 0.1, 0.5, 0.99]
fig,ax = plt.subplots(nrows=1, ncols=4, figsize=(15,4))
plt_ind = np.arange(4) + 141
for decay, ind in zip(lr_decay, plt_ind):
    plt.subplot(ind)
    learn_rate = np.exp(-epochs * decay)
    plt.plot(epochs, learn_rate, c='cyan')
    plt.title('decay rate: ' + str(decay))
    plt.xlabel('epochs $t$')
    plt.ylabel('$\\eta^(t)$')
fig.subplots_adjust(hspace=0.5, wspace=0.3)
plt.show()

In [ ]:
#The neighborhood distance function is given by:
distance = np.arange(0, 30)
sigma_sq = [0.1, 1, 10, 100]
fig,ax = plt.subplots(nrows=1, ncols=4, figsize=(15,4))
plt_ind = np.arange(4) + 141
for s, ind in zip(sigma_sq, plt_ind):
    plt.subplot(ind)
    f = np.exp(-distance ** 2 / 2 / s)
    plt.plot(distance, f, c='cyan')
    plt.title('$\\sigma^2$ = ' + str(s))
    plt.xlabel('Distance')
    plt.ylabel('Neighborhood function $f$')
fig.subplots_adjust(hspace=0.5, wspace=0.3)
plt.show()

In [ ]:
# Return the (g,h) index of the BMU in the grid
def find_BMU(SOM,x):
    distSq = (np.square(SOM - x)).sum(axis=2)
    return np.unravel_index(np.argmin(distSq, axis=None), distSq.shape)
    
# Update the weights of the SOM cells when given a single training example
# and the model parameters along with BMU coordinates as a tuple
def update_weights(SOM, train_ex, learn_rate, radius_sq, 
                   BMU_coord, step=3):
    g, h = BMU_coord
    #if radius is close to zero then only BMU is changed
    if radius_sq < 1e-3:
        SOM[g,h,:] += learn_rate * (train_ex - SOM[g,h,:])
        return SOM
    # Change all cells in a small neighborhood of BMU
    for i in range(max(0, g-step), min(SOM.shape[0], g+step)):
        for j in range(max(0, h-step), min(SOM.shape[1], h+step)):
            dist_sq = np.square(i - g) + np.square(j - h)
            dist_func = np.exp(-dist_sq / 2 / radius_sq)
            SOM[i,j,:] += learn_rate * dist_func * (train_ex - SOM[i,j,:])   
    return SOM    

import time
import progressbar

def SOM_fit(SOM, train_data, learn_rate = .1, radius_sq = 1, 
             lr_decay = .1, radius_decay = .1):    
    learn_rate_0 = learn_rate
    radius_0 = radius_sq
    rand.shuffle(train_data)      
    for train_ex in train_data:
            g, h = find_BMU(SOM, train_ex)
            SOM = update_weights(SOM, train_ex, 
                                 learn_rate, radius_sq, (g,h))
            # Update learning rate and radius
            learn_rate = learn_rate_0 * np.exp(-epoch * lr_decay)
            radius_sq = radius_0 * np.exp(-epoch * radius_decay)            
    return SOM


# Main routine for training an SOM. It requires an initialized SOM grid
# or a partially trained grid as parameter
def train_SOM(SOM, train_data, learn_rate = .1, radius_sq = 1, 
             lr_decay = .1, radius_decay = .1, epochs = 10):    
    with progressbar.ProgressBar(max_value=epochs, enable_colors=False) as bar:
        learn_rate_0 = learn_rate
        radius_0 = radius_sq
        for epoch in np.arange(0, epochs):
            bar.next()
            rand.shuffle(train_data)      
            for train_ex in train_data:
                g, h = find_BMU(SOM, train_ex)
                SOM = update_weights(SOM, train_ex, 
                                     learn_rate, radius_sq, (g,h))
            # Update learning rate and radius
            learn_rate = learn_rate_0 * np.exp(-epoch * lr_decay)
            radius_sq = radius_0 * np.exp(-epoch * radius_decay)            
    return SOM

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
# Dimensions of the SOM grid
m = 25
n = 25
# size of training examples
n_x = 50
rand = np.random.RandomState(1230)
# Initialize the training data
train_data = rand.randint(0, 255, (n_x*n_x, 3))
# Initialize the SOM randomly
SOM_init = rand.randint(0, 255, (m, n, 3)).astype(float)

bounds=(-1,-1,1,1)
train_img = hv.RGB(train_data.reshape(n_x, n_x, 3), bounds=bounds)
train_img.opts(title="Выборка",height=350,width=350, xaxis='bare',yaxis='bare') 
som_img=hv.RGB(SOM_init, bounds=bounds)
som_img.opts(title="Исходная карта весов", height=350,width=350, xaxis='bare',yaxis='bare') 
train_img+som_img

In [ ]:
import progressbar
SOM_dict={}
max_epoch = 15
SOM = SOM_init.copy()
with progressbar.ProgressBar(max_value=max_epoch, enable_colors=False) as bar:
    bar.init()
    for epoch in range(0,max_epoch):
        SOM = SOM_fit(SOM, train_data, learn_rate=0.001, radius_sq = 10)
        SOM_dict[epoch]= hv.RGB(SOM.copy(), bounds=bounds).opts(title="Карта весов",height=350,width=350, xaxis='bare',yaxis='bare') 
        bar.next()


In [ ]:
hmap = hv.HoloMap(SOM_dict, kdims=['Эпоха'])
(som_img+hmap).opts(height=400,width=100)